In [1]:
#參考資料 https://www.kaggle.com/outrunner/use-keras-to-count-sea-lions
%matplotlib inline
import time
import numpy as np
import cv2
import matplotlib.pyplot as plt
import skimage.feature
import keras
import os
from keras.models import Sequential
from keras.layers import Dense, Activation, Flatten, Conv2D, MaxPooling2D
from keras.models import load_model
from tempfile import TemporaryFile

Using TensorFlow backend.


In [2]:
r = 0.4     #scale down
width = 100 #patch size 
filedir       = '/home/alvin/cei/notebook_home/ai_lab_image/corpus/Kaggle_Sealion_10'
skip_loading  = True
Tbeg_time = time.time()

In [3]:
def GetData(filename):
    # read the Train and Train Dotted images
    #image_1 = cv2.imread("../input/TrainDotted/" + filename)
    #image_2 = cv2.imread("../input/Train/" + filename)
    image_1 = cv2.imread(filedir + '/TrainDotted/' + filename)
    image_2 = cv2.imread(filedir + '/Train/' + filename)    
    img1 = cv2.GaussianBlur(image_1,(5,5),0)

    # absolute difference between Train and Train Dotted
    if(image_1.shape !=  image_2.shape):
        #skip due to size not equal
        return np.array([]),np.array([]),'error_no_2'
    image_3 = cv2.absdiff(image_1,image_2)
    mask_1 = cv2.cvtColor(image_1, cv2.COLOR_BGR2GRAY)
    mask_1[mask_1 < 50] = 0
    mask_1[mask_1 > 0] = 255
    image_4 = cv2.bitwise_or(image_3, image_3, mask=mask_1)

    # convert to grayscale to be accepted by skimage.feature.blob_log
    image_6 = np.max(image_4,axis=2)
  
    # alvin: check blog_log
    rmse = np.sqrt((image_3**2).mean())
    if(rmse > 8.0):
        #print('skip due to blob failed')
        return np.array([]),np.array([]),'error_no_1'
    
    
    # detect blobs
    blobs = skimage.feature.blob_log(image_6, min_sigma=3, max_sigma=7, num_sigma=1, threshold=0.05)

    h,w,d = image_2.shape

    res=np.zeros((int((w*r)//width)+1,int((h*r)//width)+1,5), dtype='int16')

    for blob in blobs:
        # get the coordinates for each blob
        y, x, s = blob
        # get the color of the pixel from Train Dotted in the center of the blob
        b,g,R = img1[int(y)][int(x)][:]
        x1 = int((x*r)//width)
        y1 = int((y*r)//width)
        # decision tree to pick the class of the blob by looking at the color in Train Dotted
        if R > 225 and b < 25 and g < 25: # RED
            res[x1,y1,0]+=1
        elif R > 225 and b > 225 and g < 25: # MAGENTA
            res[x1,y1,1]+=1
        elif R < 75 and b < 50 and 150 < g < 200: # GREEN
            res[x1,y1,4]+=1
        elif R < 75 and  150 < b < 200 and g < 75: # BLUE
            res[x1,y1,3]+=1
        elif 60 < R < 120 and b < 50 and g < 75:  # BROWN
            res[x1,y1,2]+=1

    ma = cv2.cvtColor((1*(np.sum(image_1, axis=2)>20)).astype('uint8'), cv2.COLOR_GRAY2BGR)
    img = cv2.resize(image_2 * ma, (int(w*r),int(h*r)))
    h1,w1,d = img.shape

    trainX = []
    trainY = []

    for i in range(int(w1//width)):
        for j in range(int(h1//width)):
            trainY.append(res[i,j,:])
            trainX.append(img[j*width:j*width+width,i*width:i*width+width,:])

    return np.array(trainX), np.array(trainY), 'ok'
    #return trainX,trainY

def rmse(predictions, targets):
    return np.sqrt(((predictions - targets) ** 2).mean())

In [4]:
# get file list
filelist = os.listdir(filedir + '/Train')
filelist.sort()
filelist = filter(lambda x: x.endswith('.jpg'), filelist)
filelist = list(filelist)
print('Total traing file = ' + str(len(filelist)))

Total traing file = 21


In [5]:
n_train = int(len(filelist)*0.7)
trainset = np.arange(n_train)
testingset = np.arange(n_train,len(filelist))

In [6]:
#prepare training vector
final_trainX = np.array([])
final_trainY = np.array([])
final_testX  = np.array([])
final_testY  = np.array([])

if skip_loading == False:
    
    print('loading training data ...')
    
    trainset = np.arange(n_train)
    for i in trainset:
    
        trainX, trainY,flag = GetData(filelist[i])
        if(flag == 'ok'):
            print('Traing '+ str(i) + '\t' + str(filelist[i]))
        else:
            print('Traing '+ str(i) + '\t' + str(filelist[i]) + '\t' + flag)
            continue
    
        np.random.seed(1004)
        randomize = np.arange(len(trainX))
        np.random.shuffle(randomize)
        trainX = trainX[randomize]
        trainY = trainY[randomize]
    
        final_trainX = np.vstack([final_trainX, trainX]) if final_trainX.size else trainX
        final_trainY = np.vstack([final_trainY, trainY]) if final_trainY.size else trainY
    '''
    print('\nloading testing data')    
    testingset = np.arange(n_train,len(filelist))
    #testingset = np.arange(n_train,n_train+10)
    for i in testingset:
        testX, testY, flag = GetData(filelist[i])
        if(testX.size):
            print('Testing '+ str(i) + '\t' + str(filelist[i]))
        else:
            print('Tesing '+ str(i) + '\t' + str(filelist[i]) + '\t' + flag)
            continue
    
        np.random.seed(1004)
        randomize = np.arange(len(testX))
        np.random.shuffle(randomize)
        testX = testX[randomize]
        testY = testY[randomize]
    
        final_testX = np.vstack([final_testX, testX]) if final_testX.size else testX
        final_testY = np.vstack([final_testY, trainY]) if final_testY.size else trainY
    '''

# save traing/testing data to file

In [7]:
if skip_loading == False:
    np.save('o_trainx', final_trainX)
    np.save('o_trainy', final_trainY)
    #np.save('o_testx', final_testX)
    #np.save('o_testy', final_testY)

# load traing/testing data from file

In [8]:
if skip_loading == True:
    final_trainX = np.load('o_trainx.npy')
    final_trainY = np.load('o_trainy.npy')
    #final_testX = np.load('o_testx.npy')
    #final_testY = np.load('o_testy.npy')
    print('skip_loading flag is on')
    print('we hange trainX/trainY from '+str(n_train)+' file')
    print('total testing file number is ' + str(len(range(n_train,len(filelist)))))

skip_loading flag is on
we hange trainX/trainY from 14 file
total testing file number is 7


In [9]:
print(final_trainY.shape)

(3824, 5)


In [ ]:
"""
fig = plt.figure(figsize=(12,12))
for i in range(4):
    ax = fig.add_subplot(1,4,i+1)
    plt.imshow(cv2.cvtColor(trainX[i], cv2.COLOR_BGR2RGB))
print(trainY[:4])
"""

In [10]:
if skip_loading == False:
    model = Sequential()
    
    model.add(Conv2D(32, (3, 3), activation='relu', padding='same', input_shape=(width,width,3)))
    model.add(Conv2D(32, (3, 3), activation='relu', padding='same'))
    model.add(MaxPooling2D(pool_size=(2,2)))
    model.add(Conv2D(64, (3, 3), activation='relu', padding='same'))
    model.add(Conv2D(64, (3, 3), activation='relu', padding='same'))
    model.add(MaxPooling2D(pool_size=(2,2)))
    model.add(Conv2D(128, (3, 3), activation='relu', padding='same'))
    model.add(Conv2D(128, (3, 3), activation='relu', padding='same'))
    model.add(MaxPooling2D(pool_size=(2,2)))
    model.add(Flatten())
    model.add(Dense(256, activation='relu'))
    model.add(Dense(5, activation='linear'))


In [11]:
if skip_loading == False:
    start_time = time.time()
    optim = keras.optimizers.SGD(lr=1e-5, momentum=0.2)
    model.compile(loss='mean_squared_error', optimizer=optim)
    model.fit(final_trainX, final_trainY, epochs=8, verbose=2)
    end_time = time.time()
    print('\nspend ' + str(round(end_time-start_time,1))+'s')   

In [12]:
if skip_loading == False:
    start_time = time.time()
    optim = keras.optimizers.SGD(lr=1e-4, momentum=0.9)
    model.compile(loss='mean_squared_error', optimizer=optim)
    model.fit(final_trainX, final_trainY, epochs=30, verbose=2)
    end_time = time.time()
    print('\nspend ' + str(round(end_time-start_time,1))+'s')   

In [21]:
if skip_loading == False:
    model.save('cei-sealion.h5')
if skip_loading == True:
    model = keras.models.load_model('cei-sealion.h5')

# <font color=blue>training data</font> evaluation

In [ ]:
start_time = time.time()
#n_train = 20
trainset     = np.arange(n_train)
five_rmse_vec= np.array([])
dist_vec     = np.zeros((1,5))
for i in trainset:
    
    trainX, trainY, flag = GetData(filelist[i])
    if(trainX.size):
        print('Traing '+ str(i) + '\t' + str(filelist[i]))
    else:
        print('Traing '+ str(i) + '\t' + str(filelist[i]) + '\t' + flag)
        continue
        
    result = model.predict(trainX)
    ground_truth_vec = np.sum(trainY, axis=0)
    prediction_vec   = np.sum(result*(result>0.3), axis=0).astype('int')
    five_class_rmse  = rmse(prediction_vec,ground_truth_vec)
    five_rmse_vec    = np.append(five_rmse_vec,five_class_rmse)
    dist_vec        += np.abs(prediction_vec-ground_truth_vec)
    
    
    print('Training set --')
    print('    ground truth: ', ground_truth_vec)
    print('  evaluate count: ', prediction_vec)
    print('      difference: ', str(np.abs(prediction_vec-ground_truth_vec)))
    print('            rmse: ', str(five_class_rmse))

print('\nfinal rmse: ',str(np.average(five_rmse_vec)))
print('final diffence ', str(dist_vec))
end_time = time.time()
print('\nspend ' + str(round(end_time-start_time,1))+'s')    

Traing 0	0.jpg
Training set --
    ground truth:  [ 58  12 472  41 330]
  evaluate count:  [ 64  55 193  82 105]
      difference:  [  6  43 279  41 225]
            rmse:  162.500461538
Traing 1	1.jpg
Training set --
    ground truth:  [ 2 20  0 11  0]
  evaluate count:  [40 38 99 54 40]
      difference:  [38 18 99 43 40]
            rmse:  54.8051092509
Traing 2	10.jpg
Training set --
    ground truth:  [4 4 7 5 0]
  evaluate count:  [27 21 60 48 37]
      difference:  [23 17 53 43 37]
            rmse:  37.0
Traing 3	11.jpg
Training set --
    ground truth:  [ 3  5 36 13  0]
  evaluate count:  [23 37 85 40 44]
      difference:  [20 32 49 27 44]
            rmse:  36.0277670693
Traing 4	12.jpg
Training set --
    ground truth:  [ 3  9 13  1  7]
  evaluate count:  [15 55 51 44 17]
      difference:  [12 46 38 43 10]
            rmse:  33.6243959054


# <font color=red>testing data</font> evaluation

In [ ]:
start_time = time.time()
#trainset      = np.arange(n_train)
five_rmse_vec= np.array([])
dist_vec     = np.zeros((1,5))
testingset = np.arange(n_train,len(filelist))
for i in testingset:
    
    testX, testY, flag = GetData(filelist[i])
    if(testX.size):
        print('Testing '+ str(i) + '\t' + str(filelist[i]))
    else:
        print('Testing '+ str(i) + '\t' + str(filelist[i]) + '\t' + flag)
        continue
        
    result = model.predict(testX)
    ground_truth_vec = np.sum(testY, axis=0)
    prediction_vec   = np.sum(result*(result>0.3), axis=0).astype('int')
    five_class_rmse  = rmse(prediction_vec,ground_truth_vec)
    five_rmse_vec    = np.append(five_rmse_vec,five_class_rmse)
    dist_vec        += np.abs(prediction_vec-ground_truth_vec)
    
    
    print('Testing set --')
    print('    ground truth: ', ground_truth_vec)
    print('  evaluate count: ', prediction_vec)
    print('      difference: ', str(np.abs(prediction_vec-ground_truth_vec)))
    print('            rmse: ', str(five_class_rmse))

print('\nfinal rmse: ',str(np.average(five_rmse_vec)))
print('final diffence ', str(dist_vec))
end_time = time.time()
print('\nspend ' + str(end_time-start_time)+'s')    

In [ ]:
Tend_time = time.time()
print('\nspend ' + str(round(Tend_time-Tbeg_time,1))+'s') 